In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd

### Constant Declaration

#### Global

In [2]:
stock_number = 100
minimum_score = 50

upper_th = {
    "rsi" : 70,
    "10di-" : 18,
    "25di-" : 18,
}

lower_th = {
    "10di+" : 25,
    "25di+":25,
    "25adx":20,
    "10adx":25,
}

#### Option : TradingView

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Mobile Safari/537.36'}

tv_url = "https://id.tradingview.com/markets/stocks-indonesia/market-movers-active/"
tv_index_class = "apply-common-tooltip tickerNameBox-GrtoTeat tickerName-GrtoTeat"

yh_url = "https://finance.yahoo.com/quote/"

### Acquiring Stock Index

#### Option : TradingView

In [ ]:
def fetch_index():
    tv_page = req.get(tv_url,headers=headers)
    soup = BeautifulSoup(tv_page.content, 'html.parser')
    
    main_table = soup.find('table')
    stock_rows = main_table.find_all('a',class_="apply-common-tooltip tickerNameBox-GrtoTeat tickerName-GrtoTeat")
    
    return [row.get_text() for row in stock_rows]

#### Option : IDX List

In [ ]:
def read_index():
    stocks = pd.read_csv("stock_list.csv",delimiter=";")
    return stocks['Kode'].tolist()

#### Implementation

In [ ]:
get_stock_index = {"TV":fetch_index, "IDX":read_index}

In [ ]:
stocks = get_stock_index['TV']

### Stock Price Mining

#### Global

##### Fetching Functions Declaration

In [6]:
def convert_text_to_float(text):
    return float(text.replace(",",""))

def parse_price_per_row(columns):
    return [convert_text_to_float(col.get_text()) for col in columns]

In [7]:
def fetch_stock_rows(stock):
    yh_page = req.get(yh_url+stock+".JK/history",headers=headers)
    soup = BeautifulSoup(yh_page.content, 'html.parser')
    
    main_table = soup.find('table')
    price_rows = main_table.find_all('tr')[1:50]
    
    return price_rows

def get_stock_df(price_rows):
    data_rows = []
    
    for row in price_rows:
        columns = row.find_all('td')
        columns = columns[1:5]
    
        try : 
            open, high, low, close = parse_price_per_row(columns)
            data_rows.append({'Open':open,'High':high,'Low':low,'Close':close})
        except :
            pass

    df = pd.DataFrame(data_rows)
    reversed_df = df.iloc[::-1].reset_index(drop=True)
        
    return reversed_df

##### Assessment Functions Declaration

In [8]:
def get_true_range(df):
    tr_1 = df['High'] - df['Low']
    tr_2 = abs(df['High'] - df['Close'].shift())
    tr_3 = abs(df['Low'] - df['Close'].shift())
    
    tr_final = pd.concat([tr_1,tr_2,tr_3],axis=1).max(axis=1)
    return tr_final

In [9]:
def get_rsi(df, period=14):
    delta = df['Close'].diff()
    
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        
    avg_gain = gain.ewm(com=period-1, min_periods=period).mean()
    avg_loss = loss.ewm(com=period-1, min_periods=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
        
    return rsi

def get_atr(df, period=14):
    tr = get_true_range(df)
    return tr.rolling(window=period).mean()

def get_dis(df, period=14):
    atr = get_atr(df, period)
    
    up = df['High'] - df['High'].shift()
    down = df['Low'].shift() - df['Low']
    
    plus_dm = up.where((up > down) & (up > 0), 0)
    minus_dm = down.where((down > up) & (down > 0), 0)
    
    plus_dm_smooth = plus_dm.ewm(alpha=1/period, min_periods=period).mean()
    minus_dm_smooth = minus_dm.ewm(alpha=1/period, min_periods=period).mean()
    
    plus_di = 100 * (plus_dm_smooth / atr)
    minus_di = 100 * (minus_dm_smooth / atr)
    
    return plus_di, minus_di

def get_adx(df, period=14):
    plus_di, minus_di = get_dis(df, period)
    
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    
    adx = dx.ewm(alpha=1/period, min_periods=period).mean()
        
    return adx


In [10]:
def assess_rsi(df,period=14):
    rsi = get_rsi(df, period)
    return rsi.iloc[-1] <= upper_th['rsi']

def assess_dis(df,period=14):
    plus_di, minus_di = get_dis(df, period)
    return plus_di.iloc[-1] >= lower_th['10di+'] and minus_di.iloc[-1] <= upper_th['10di-']

def assess_adx(df,period=14):
    adx = get_adx(df, period)
    
    th = lower_th['10adx'] if period < 18 else lower_th['25adx']
    return adx.iloc[-1] >= th

def assess_stock(df):    
    rsi = assess_rsi(df)
    
    short_dis = assess_dis(df,period=10)
    long_dis = assess_dis(df,period=25)
    
    short_adx = assess_adx(df,period=10)
    long_adx = assess_adx(df,period=25)
    
    return rsi, short_dis, long_dis, short_adx, long_adx

In [11]:
def export_passed_stock(stock,score,assessment):
    return {"Stock":stock,"Score":score,"RSI":bool(assessment[0]),"10DI":bool(assessment[1]),"25DI":bool(assessment[2]),"10ADX":bool(assessment[3]),"25ADX":bool(assessment[4])}

####  Implementation

In [12]:
passed_stock = []

for stock in stocks : 
    if len(passed_stock) >= stock_number:
        break
    
    stock_price_rows = fetch_stock_rows(stock)
    stock_df = get_stock_df(stock_price_rows)
        
    
    try:
        assessment = assess_stock(stock_df)

        score = (sum(assessment)/len(assessment))*100
        if score >= minimum_score:
            new_line = export_passed_stock(stock,score,assessment)
            passed_stock.append(new_line)
            
    except:
        pass
    
final_df = pd.DataFrame(passed_stock)

In [13]:
final_df = final_df.sort_values(by=['Score'],ascending=False)
final_df.to_csv("stock_assessment.csv",index=False)